In [1]:
pip install langgraph langchain langchain-community langchain-openai python-dotenv

     -------------------------------------- 118.0/118.0 kB 6.7 MB/s eta 0:00:00
     ---------------------------------------- 1.0/1.0 MB 9.0 MB/s eta 0:00:00
     ---------------------------------------- 2.4/2.4 MB 9.6 MB/s eta 0:00:00
     ---------------------------------------- 50.4/50.4 kB 2.5 MB/s eta 0:00:00
  Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
     -------------------------------------- 408.7/408.7 kB 6.4 MB/s eta 0:00:00
     -------------------------------------- 434.9/434.9 kB 6.7 MB/s eta 0:00:00
     -------------------------------------- 302.2/302.2 kB 4.7 MB/s eta 0:00:00
     -------------------------------------- 884.8/884.8 kB 4.3 MB/s eta 0:00:00
     -------------------------------------- 387.1/387.1 kB 4.8 MB/s eta 0:00:00
     ---------------------------------------- 49.5/49.5 kB 2.5 MB/s eta 0:00:00
  Using cached packaging-24.1-py3-none-any.whl (53 kB)
     ---------------------------------------- 74.8/74.8 kB 2.1 MB/s eta 0:00:00
     ----

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sktime 0.21.1 requires numpy<1.26,>=1.21.0, but you have numpy 1.26.4 which is incompatible.
pycaret 3.2.0 requires scipy~=1.10.1, but you have scipy 1.13.1 which is incompatible.


In [7]:
import os
from dotenv import load_dotenv
load_dotenv('.env')

WEATHER_API_KEY = os.environ['WEATHER_API_KEY']
TAVILY_API_KEY = os.environ['TAVILY_API_KEY']
TOGETHER_API_KEY = os.environ['TOGETHER_API_KEY']

In [8]:
import requests
from typing import List, Literal
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI

In [9]:
@tool
def get_weather(query: str) -> list:
    """Search weatherapi to get the current weather"""
    endpoint = f"http://api.weatherapi.com/v1/current.json?key={WEATHER_API_KEY}&q={query}"
    response = requests.get(endpoint)
    data = response.json()

    if data.get("location"):
        return data
    else:
        return "Weather Data Not Found"

@tool
def search_web(query: str) -> list:
    """Search the web for a query"""
    tavily_search = TavilySearchResults(api_key=TAVILY_API_KEY, max_results=2, search_depth='advanced', max_tokens=1000)
    results = tavily_search.invoke(query)
    return results

In [12]:

# Initialize the ChatOpenAI instance with Together API details
llm = ChatOpenAI(
    base_url="https://api.together.xyz/v1",
    api_key=TOGETHER_API_KEY,
    model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo"
)

# Alternative: Initialize with OpenAI key if available
# llm = ChatOpenAI(model="gpt-4o-mini", api_key="sk-U7tijaa4jwHvhVWGr....", temperature=0)

# Define tools
tools = [search_web, get_weather]

# Bind tools to LLM
llm_with_tools = llm.bind_tools(tools)


In [13]:
prompt = """
    Given only the tools at your disposal, mention tool calls for the following tasks:
    Do not change the query given for any search tasks
        1. What is the current weather in Trivandrum today
        2. Can you tell me about Kerala
        3. Why is the sky blue?
    """

results = llm_with_tools.invoke(prompt)

print(results.tool_calls)

query = "What is the current weather in Trivandrum today"
response = llm.invoke(query)
print(response.content)

[{'name': 'search_web', 'args': {'query': 'Current weather in Trivandrum today'}, 'id': 'call_w5gzmxyq9xv2c9d4ymq75n4s', 'type': 'tool_call'}, {'name': 'search_web', 'args': {'query': 'Kerala'}, 'id': 'call_8mmk1dzz7yrxj7iuxr2gjrr8', 'type': 'tool_call'}, {'name': 'search_web', 'args': {'query': 'Why is the sky blue?'}, 'id': 'call_0xeqv4vwsa2q07imr9grv7ox', 'type': 'tool_call'}]
I'm not able to access real-time information about the current weather in Trivandrum. However, I can suggest some options to find out the current weather in Trivandrum.

You can check the current weather in Trivandrum, now Thiruvananthapuram, by visiting a weather website such as:

1. AccuWeather (accuweather.com)
2. Weather.com (weather.com)
3. World Weather Online (worldweatheronline.com)
4. Indian Meteorological Department (imd.gov.in)

Please note that these websites may not provide the most up-to-date information, and the current weather may be different from the forecast. 

If you can provide more contex

In [14]:
from langgraph.prebuilt import create_react_agent

# system prompt is used to inform the tools available to when to use each
system_prompt = """Act as a helpful assistant.
    Use the tools at your disposal to perform tasks as needed.
        - get_weather: whenever user asks get the weather of a place.
        - search_web: whenever user asks for information on current events or if you don't know the answer.
    Use the tools only if you don't know the answer.
    """

# we can initialize the agent using the llama3 model, tools, and system prompt.
agent = create_react_agent(model=llm, tools=tools, state_modifier=system_prompt)

# Let’s query the agent to see the result.
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

inputs = {"messages": [("user", "What is the current weather in Trivandrum today")]}

print_stream(agent.stream(inputs, stream_mode="values"))

================================ Human Message =================================

What is the current weather in Trivandrum today
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_ygp4p6z7n6v583gofbmgx2e4)
 Call ID: call_ygp4p6z7n6v583gofbmgx2e4
  Args:
    query: Trivandrum
================================= Tool Message =================================
Name: get_weather

{"location": {"name": "Trivandrum", "region": "Kerala", "country": "India", "lat": 8.5069, "lon": 76.9569, "tz_id": "Asia/Kolkata", "localtime_epoch": 1730460175, "localtime": "2024-11-01 16:52"}, "current": {"last_updated_epoch": 1730459700, "last_updated": "2024-11-01 16:45", "temp_c": 28.4, "temp_f": 83.1, "is_day": 1, "condition": {"text": "Mist", "icon": "//cdn.weatherapi.com/weather/64x64/day/143.png", "code": 1030}, "wind_mph": 7.6, "wind_kph": 12.2, "wind_degree": 286, "wind_dir": "WNW", "pressure_mb": 1009.0, "pressure_in": 29.8, "precip_mm": 0.0

In [16]:
print("_________________________________________")
print("Custom Agent")

_________________________________________
Custom Agent


In [31]:
# import the required methods
from langgraph.prebuilt import ToolNode
from langgraph.graph import StateGraph, MessagesState, START, END

# define a tool_node with the available tools
tools = [search_web, get_weather]
tool_node = ToolNode(tools)

# define functions to call the LLM or the tools
def call_model(state: MessagesState):
    messages = state["messages"]
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

def call_tools(state: MessagesState) -> Literal["tools", END]:
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END

# initialize the workflow from StateGraph
workflow = StateGraph(MessagesState)

# add a node named LLM, with call_model function. This node uses an LLM to make decisions based on the input given
workflow.add_node("LLM", call_model)

# Our workflow starts with the LLM node
workflow.add_edge(START, "LLM")

# Add a tools node
workflow.add_node("tools", tool_node)

# Add a conditional edge from LLM to call_tools function. It can go tools node or end depending on the output of the LLM. 
workflow.add_conditional_edges("LLM", call_tools)

# tools node sends the information back to the LLM
workflow.add_edge("tools", "LLM")

agent = workflow.compile()
# display(Image(agent.get_graph().draw_mermaid_png()))

for chunk in agent.stream(
    {"messages": [("user", "Will it rain in Bangalore, India today?")]},
    stream_mode="values",):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

Will it rain in Bangalore, India today?
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_lxnl8yxjlo6c212341o5fdke)
 Call ID: call_lxnl8yxjlo6c212341o5fdke
  Args:
    query: Bangalore, India
================================= Tool Message =================================
Name: get_weather

{"location": {"name": "Bangalore", "region": "Karnataka", "country": "India", "lat": 12.9833, "lon": 77.5833, "tz_id": "Asia/Kolkata", "localtime_epoch": 1730484688, "localtime": "2024-11-01 23:41"}, "current": {"last_updated_epoch": 1730484000, "last_updated": "2024-11-01 23:30", "temp_c": 20.6, "temp_f": 69.1, "is_day": 0, "condition": {"text": "Clear", "icon": "//cdn.weatherapi.com/weather/64x64/night/113.png", "code": 1000}, "wind_mph": 5.4, "wind_kph": 8.6, "wind_degree": 73, "wind_dir": "ENE", "pressure_mb": 1014.0, "pressure_in": 29.95, "precip_mm": 